まずざっくりデータを見るという方法を試す。参照はこちら。

[ALBERT Office blog, 高次元データの可視化の手法をSwiss rollを例に見てみよう](http://blog.albert2005.co.jp/2014/12/11/%E9%AB%98%E6%AC%A1%E5%85%83%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%AE%E6%89%8B%E6%B3%95%E3%82%92swiss-roll%E3%82%92%E4%BE%8B%E3%81%AB%E8%A6%8B%E3%81%A6%E3%81%BF%E3%82%88/)

RDRToolbox の準備が必要だが、これは別途やっておく。参照元の source と biocLite が相当する。

In [ ]:
#RDRToolbox を利用
library(RDRToolbox)

#Swiss Roll を作成
N=1000
p = sqrt(2 + 2 * seq(-1, 1 - 2/N, 2/N))
y = 2 * runif(N, -1, 1)
d_sr = cbind(p * cos(2*pi*p), y, p * sin(2*pi*p))
 
color <- vector()
for(i in 1:N){
 color[i] = rainbow(ceiling(N/50))[ceiling(i/50)]
}
labels=seq(N)

#見てみる
#plotDR(data=d_sr, labels=labels, col=color)

まず古典的らしい、高次元データの可視化。３つあるらしい。

- 主成分分析（PCA）
- 多次元尺度構成法（MDS)
- 自己組織化マップ（SOM）

## 主成分分析(PCA)

In [ ]:
#主成分分析による次元削除
d_pca <- prcomp(d_sr, scale=TRUE)
d_pca$x[,1:2]
plotDR(data=d_pca$x[,1:2], labels=labels, col=color)

## 多次元尺度構成法(MDS)

In [ ]:
#多次元尺度構成法による次元削除
d_cmds <- cmdscale(dist(d_sr), k = 2)
plotDR(data=d_cmds, labels=labels, col=color)

## 自己組織化マップ(SOM)

In [ ]:
#自己組織化マップによる可視化
install.packages("som") #somパッケージのインストール
library(som)
d_som <- som(normalize(d_sr), xdim=8, ydim=4)
plotDR(data=(d_som$visual[,1:2]+0.5+cbind(rnorm(nrow(d_sr),0, 0.2), rnorm(nrow(d_sr), 0, 0.2))), labels=labels, col=color)

古典的な方法でうまく行かない場合、改良されている手法を試す。

## カーネル主成分分析
データが非線形だと主成分分析はうまくいかない。

In [ ]:
#カーネル主成分分析による次元削除
install.packages("kernlab") #kernlabパッケージのインストール
library(kernlab)
d_kpca <- kpca(d_sr, kernel = "rbfdot", features=2, kpar=list(sigma=3))
plotDR(data=pcv(d_kpca), labels=labels, col=color)

## Isomap
多次元尺度構成法のデータ同士の近さを測る方法として、ユークリッド空間を一般化させた多様体を用いるやり方。結構時間が掛かる。

In [ ]:
#isomapによる次元削除
#library(RDRToolbox)
d_isomap <- Isomap(data=d_sr, dims=2, k=5)
plotDR(data=d_isomap[[1]], labels=labels, col=color)

## Locally Linear Embedding(LLE)
isomap以外の多様体学習として、多様体が局所的にユークリッドであることを用いる。

lleパッケージのインストールは別途やっておく。

In [ ]:
#Locally Linear Embeddingによる次元削除
#install.packages("lle") #lleパッケージのインストール
library(lle)
d_lle <- lle(d_sr, m=2, k=12)
plotDR(data=d_lle[[1]], labels=labels, col=color)

## Diffusion Map
多様体上のランダムウォークを利用する。

diffutionMapパッケージのインストールは別途やっておく。

In [ ]:
#diffusion mapによる次元削除
#install.packages("diffusionMap") #diffusionMapパッケージのインストール
library(diffusionMap)
d_dffmap <- diffuse(dist(scale(d_sr)))
plotDR(data=d_dffmap$X[,1:2], labels=labels, col=color)